In [ ]:
# 1. Collega Google Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
# 2. Copia il progetto nella RAM locale di Colab
# ATTENZIONE: Questo copierà anche i VIDEO. Se sono troppi GB, potrebbe riempire lo spazio.
# Se hai problemi di spazio, copia solo le cartelle necessarie.

import shutil
import os
import time
from tqdm.notebook import tqdm

# Percorso sorgente (Drive)
source_dir = '/content/drive/My Drive/human-robot-collaboration'
# Percorso destinazione (Locale Colab)
dest_dir = '/content/human-robot-collaboration'

print(f"Inizio copia da {source_dir} a {dest_dir}...")

if os.path.exists(dest_dir):
    print("Rimozione vecchia cartella locale...")
    shutil.rmtree(dest_dir)

# Funzione per copiare con progress bar
def copy_with_progress(src, dst):
    print("Conteggio file in corso...")
    total_files = 0
    for root, dirs, files in os.walk(src):
        total_files += len(files)
    
    print(f"Trovati {total_files} file da copiare.")
    
    with tqdm(total=total_files, unit='file', desc='Copia file') as pbar:
        for root, dirs, files in os.walk(src):
            rel_path = os.path.relpath(root, src)
            dest_root = os.path.join(dst, rel_path)
            os.makedirs(dest_root, exist_ok=True)
            
            for file in files:
                # Ignora file di Google Drive (gdoc, gsheet, ecc.) che non si possono copiare
                if file.endswith(('.gdoc', '.gsheet', '.gslides')):
                    pbar.update(1)
                    continue

                src_file = os.path.join(root, file)
                dst_file = os.path.join(dest_root, file)
                shutil.copy2(src_file, dst_file)
                pbar.update(1)

start_time = time.time()
copy_with_progress(source_dir, dest_dir)
end_time = time.time()

print(f"Copia completata in {end_time - start_time:.2f} secondi!")

# Spostiamoci nella cartella locale
os.chdir(dest_dir)
print(f"Working directory impostata su: {os.getcwd()}")

In [ ]:
# 3. Installa le dipendenze
!pip install -r requirements.txt

In [ ]:
# 4. Avvia il Training del Pose Estimator
# --stride 10 significa che usa 1 frame ogni 10 per velocizzare il training
# --batch_size 32 è standard, riduci se finisci la memoria GPU
# --num_workers 8 parallelizza il caricamento dei video (essenziale per velocità)

!python train_pose_estimator.py --epochs 20 --batch_size 32 --stride 10 --num_workers 8

In [ ]:
# 5. Salva il modello allenato su Drive
import shutil
import os

local_checkpoint = '/content/human-robot-collaboration/checkpoints/pose_estimator_best.pth'
drive_checkpoint_dir = '/content/drive/My Drive/human-robot-collaboration/checkpoints'

if os.path.exists(local_checkpoint):
    print(f"Salvataggio modello su Drive in {drive_checkpoint_dir}...")
    os.makedirs(drive_checkpoint_dir, exist_ok=True)
    shutil.copy2(local_checkpoint, drive_checkpoint_dir)
    print("Modello salvato con successo!")
else:
    print("Nessun modello trovato. Il training è fallito?")